In [1]:
import os
import sys
import settings

initial_path = set(sys.path)
sys.path.append(settings.CARLA_EGG_PATH)

# ADD 
try:
    sys.path.append(os.path.abspath('.') + '/PythonAPI/carla')
except IndexError:
    pass

new_paths = set(sys.path) - initial_path
for path in new_paths:
    print(f"Added: {path} to the Path")

Added: /home/rudy/Documents/carla-dataset-runner/PythonAPI/carla to the Path
Added: /home/rudy/Documents/carla-dataset-runner/PythonAPI/carla/dist/carla-0.9.11-py3.7-linux-x86_64.egg to the Path


In [16]:
import h5py
import numpy as np
import cv2
from carla import ColorConverter

In [59]:
def depth2grayscale(arr):
    arr[arr == 1000] = 0
    normalized_depth = cv2.normalize(arr, arr, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    normalized_depth = np.stack((normalized_depth,)*3, axis=-1)  # Grayscale into 3 channels
    return normalized_depth


def labels_to_cityscapes_palette(arr):
    classes = {
        0: [0, 0, 0],         # None
        1: [70, 70, 70],      # Buildings
        2: [100, 40, 40],     # Fences
        3: [55, 90, 80],      # Other
        4: [220, 20, 60],     # Pedestrians
        5: [153, 153, 153],   # Poles
        6: [157, 234, 50],    # RoadLines
        7: [128, 64, 128],    # Roads
        8: [244, 35, 232],    # Sidewalks
        9: [107, 142, 35],    # Vegetation
        10: [0, 0, 142],      # Vehicles
        11: [102, 102, 156],  # Walls
        12: [220, 220, 0],    # TrafficSigns
        13: [70, 130, 180],   # Sky
        14: [81, 0, 81],      # Ground
        15: [150, 100, 100],  # Bridge
        16: [230, 150, 140],  # RailTrack
        17: [180, 165, 180],  # GuardRail
        18: [250, 170, 30],   # Traffic Light
        19: [110, 190, 160],  # Static
        20: [170, 120, 50],   # Dynamic
        21: [45, 60, 150],    # Water
        22: [145, 170, 100]   # Terrain
    }
    
    result = np.zeros((arr.shape[0], arr.shape[1], 3))
    for key, value in classes.items():
        result[np.where(arr == key)] = value
        
    result /= 255.0
    return result


In [65]:
file_path = "data/test3.hdf5"
f = h5py.File(file_path, "r")
total_run = sum([k.startswith("run") for k in f.keys()])
print(f"The file: {file_path} contains {total_run} ego runs")

The file: data/test3.hdf5 contains 5 ego runs


In [66]:
run = f['run_000_morning']
run_rgb = run['rgb']
run_depth = run['depth']
run_semantic = run['semantic']

In [70]:
timestamps = list(run_rgb.keys())
print(f"{len(timestamps)} frames for selected record")

150 frames for selected record


In [71]:
for timestamp in timestamps:
    rgb_image = np.array(run_rgb[timestamp])
    depth_map = np.array(run_depth[timestamp])
    semantic_map = np.array(run_semantic[timestamp])
    cv2.imshow('rgb', rgb_image)
    cv2.imshow('depth', depth2grayscale(depth_map))
    cv2.imshow('semantic', labels_to_cityscapes_palette(semantic_map))
    cv2.waitKey(50)
cv2.destroyAllWindows()